In [54]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

In [55]:
options_full = pd.read_csv('/Users/landlee/Desktop/Option Metrics to Share/opt_full_list_mm.csv')

/var/folders/b1/g827t6qn66l1htcz2y_jj7640000gn/T/ipykernel_11211/2831764356.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  options_full = pd.read_csv('/Users/landlee/Desktop/Option Metrics to Share/opt_full_list_mm.csv')


In [56]:
options_full['adj_best_bid'] = options_full['best_bid'] / options_full['cfadj']
options_full['adj_best_offer'] = options_full['best_offer'] / options_full['cfadj']
options_full['strike_price'] = options_full['strike_price'] / 1000

In [57]:
stock_data = pd.read_csv("StockData.csv")

In [58]:
stock_data

,permno,date,ret,prc,vol,shrout,cfacpr,cfacshr,ticker,adj_prc,adj_ret,adj_vol,adj_shrout,mktcap
0,14593,2018-01-02,0.017905,172.25999,25555934.0,5087056.0,4.0,4.0,AAPL,43.064997,0.017905,25555934.0,20348224.0,8.762962e+08
1,14593,2018-01-03,-0.000174,172.23000,29517900.0,5087056.0,4.0,4.0,AAPL,43.057500,-0.000174,29517900.0,20348224.0,8.761437e+08
2,14593,2018-01-04,0.004645,173.03000,22434597.0,5087056.0,4.0,4.0,AAPL,43.257500,0.004645,22434597.0,20348224.0,8.802133e+08
3,14593,2018-01-05,0.011385,175.00000,23660018.0,5087056.0,4.0,4.0,AAPL,43.750000,0.011385,23660018.0,20348224.0,8.902348e+08
4,14593,2018-01-08,-0.003714,174.35001,20567766.0,5087056.0,4.0,4.0,AAPL,43.587502,-0.003714,20567766.0,20348224.0,8.869283e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24279,20067,2022-12-23,0.006550,4.61000,19309959.0,927938.0,1.0,1.0,SOFI,4.610000,0.006550,19309959.0,927938.0,4.277794e+06
24280,20067,2022-12-27,-0.047722,4.39000,24002624.0,927938.0,1.0,1.0,SOFI,4.390000,-0.047722,24002624.0,927938.0,4.073648e+06
24281,20067,2022-12-28,-0.004556,4.37000,34871669.0,927938.0,1.0,1.0,SOFI,4.370000,-0.004556,34871669.0,927938.0,4.055089e+06
24282,20067,2022-12-29,0.050343,4.59000,33061823.0,927938.0,1.0,1.0,SOFI,4.590000,0.050343,33061823.0,927938.0,4.259235e+06


In [59]:
# --- 0. Make working copies so originals are untouched ---
opts = options_full.copy()
stock = stock_data.copy()

# --- 1. Preprocess / types ---
# Dates
opts['date'] = pd.to_datetime(opts['date'])
opts['exdate'] = pd.to_datetime(opts['exdate'])
stock['date'] = pd.to_datetime(stock['date'])

# Uppercase tickers and optionally reduce memory
opts['ticker'] = opts['ticker'].astype(str).str.upper().astype('category')
stock['ticker'] = stock['ticker'].astype(str).str.upper().astype('category')

In [60]:
# --- 2. Realized volatility on underlying (30d annualized) ---
stock['log_ret'] = np.log1p(stock['adj_ret'].fillna(0))
# rolling std of log returns per ticker
stock['realized_vol_30'] = (
    stock.groupby('ticker')['log_ret']
         .transform(lambda x: x.rolling(30, min_periods=10).std())
    * np.sqrt(252)
)
stock.drop(columns=['log_ret'], inplace=True)

/var/folders/b1/g827t6qn66l1htcz2y_jj7640000gn/T/ipykernel_11211/1797868234.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  stock.groupby('ticker')['log_ret']


In [61]:
# --- 3. Filter options to desired DTE window (28–35) ---
opts['DTE'] = (opts['exdate'] - opts['date']).dt.days
opts_subset = opts.loc[(opts['DTE'] >= 28) & (opts['DTE'] <= 35)].copy()

In [62]:
opts[(opts['ticker'] == 'AAPL') & (opts['date'] == '2018-01-02') & (opts['exdate'] == '2018-02-02')]

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,root,suffix,ticker,date_year,date_month,exyear,exmonth,adj_best_bid,adj_best_offer,DTE
1521392,101594.0,2018-01-02,AAPL 180202C144000,1,2018-02-02,NaN,C,144.0,28.25,28.95,...,NaN,NaN,AAPL,2018,1,2018,2,28.25,28.95,31
1521393,101594.0,2018-01-02,AAPL 180202C145000,1,2018-02-02,NaN,C,145.0,27.25,28.00,...,NaN,NaN,AAPL,2018,1,2018,2,27.25,28.00,31
1521394,101594.0,2018-01-02,AAPL 180202C146000,1,2018-02-02,01/02/2018,C,146.0,26.25,27.00,...,NaN,NaN,AAPL,2018,1,2018,2,26.25,27.00,31
1521395,101594.0,2018-01-02,AAPL 180202C147000,1,2018-02-02,NaN,C,147.0,25.35,26.10,...,NaN,NaN,AAPL,2018,1,2018,2,25.35,26.10,31
1521396,101594.0,2018-01-02,AAPL 180202C148000,1,2018-02-02,NaN,C,148.0,24.35,25.10,...,NaN,NaN,AAPL,2018,1,2018,2,24.35,25.10,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521449,101594.0,2018-01-02,AAPL 180202P200000,1,2018-02-02,NaN,P,200.0,27.50,28.30,...,NaN,NaN,AAPL,2018,1,2018,2,27.50,28.30,31
1521450,101594.0,2018-01-02,AAPL 180202P202500,1,2018-02-02,NaN,P,202.5,29.95,30.80,...,NaN,NaN,AAPL,2018,1,2018,2,29.95,30.80,31
1521451,101594.0,2018-01-02,AAPL 180202P205000,1,2018-02-02,NaN,P,205.0,32.45,33.30,...,NaN,NaN,AAPL,2018,1,2018,2,32.45,33.30,31
1521452,101594.0,2018-01-02,AAPL 180202P207500,1,2018-02-02,NaN,P,207.5,34.95,35.75,...,NaN,NaN,AAPL,2018,1,2018,2,34.95,35.75,31


In [63]:
# --- 4. Merge underlying info needed for ATM selection ---
# Need adj_prc (spot) and cfacpr to adjust strikes
merged = opts_subset.merge(
    stock[['ticker', 'date', 'adj_prc', 'cfacpr']],
    on=['ticker', 'date'],
    how='inner',
    validate='many_to_one',
)

In [64]:
# --- 5. Compute adjusted strike and distance to spot ---
# Adjust raw strike_price by split/dividend factor to align with adjusted price
merged['adj_strike'] = merged['strike_price'] / merged['cfacpr']
merged['strike_diff'] = (merged['adj_strike'] - merged['adj_prc']).abs()

In [65]:
# --- 6. Canonicalize strike to avoid float mismatch and split legs ---
merged['adj_strike_rounded'] = merged['adj_strike'].round(4)  # adjust precision as needed

calls = merged[merged['cp_flag'].str.upper().isin(['C', 'CALL'])].copy()
puts  = merged[merged['cp_flag'].str.upper().isin(['P', 'PUT'])].copy()

In [66]:
# --- 7. Form all contemporaneous straddle observations ---
straddle_obs = calls.merge(
    puts,
    on=['ticker', 'date', 'exdate', 'adj_strike_rounded'],
    suffixes=('_call', '_put'),
    how='inner'
)

In [67]:
# --- 8. Compute premiums / basic derived fields ---
straddle_obs['call_premium'] = (straddle_obs['adj_best_bid_call'] + straddle_obs['adj_best_offer_call']) / 2
straddle_obs['put_premium']  = (straddle_obs['adj_best_bid_put']  + straddle_obs['adj_best_offer_put'])  / 2
straddle_obs['straddle_premium'] = straddle_obs['call_premium'] + straddle_obs['put_premium']
straddle_obs['net_delta'] = straddle_obs.get('delta_call', 0) + straddle_obs.get('delta_put', 0)
straddle_obs['total_open_interest'] = straddle_obs['open_interest_call'] + straddle_obs['open_interest_put']

In [68]:
# --- 9. ATMness: distance to spot ---
# underlying spot price should be identical for call/put side; extract one
if 'adj_prc_call' in straddle_obs.columns:
    spot = straddle_obs['adj_prc_call']
elif 'adj_prc_put' in straddle_obs.columns:
    spot = straddle_obs['adj_prc_put']
else:
    spot = straddle_obs['adj_prc']  # fallback if not suffixed

straddle_obs['strike_diff'] = (straddle_obs['adj_strike_rounded'] - spot).abs()

In [69]:
# --- 10. Select one ATM straddle per group with tie-breaker on open interest ---
# minimal strike_diff
straddle_obs['min_diff'] = straddle_obs.groupby(['ticker', 'date', 'exdate'])['strike_diff'].transform('min')
atm_candidates = straddle_obs.loc[straddle_obs['strike_diff'] == straddle_obs['min_diff']].copy()

# tie-break by total open interest
atm_candidates['oi_rank'] = atm_candidates.groupby(['ticker', 'date', 'exdate'])['total_open_interest'] \
                                         .rank(method='first', ascending=False)
atm_final = atm_candidates.loc[atm_candidates['oi_rank'] == 1].copy()

# clean up helper cols
atm_final.drop(columns=['min_diff', 'oi_rank'], inplace=True)

In [70]:
# --- 11. Optional liquidity filter ---
liq_mask = (
    (atm_final['open_interest_call'] >= 100) &
    (atm_final['open_interest_put'] >= 100) &
    (atm_final['volume_call'] >= 10) &
    (atm_final['volume_put'] >= 10)
)
atm_final_liq = atm_final.loc[liq_mask].copy()


In [71]:
# --- 12. Aggregate IV and tidy output ---
atm_final_liq['straddle_iv'] = (atm_final_liq['impl_volatility_call'] + atm_final_liq['impl_volatility_put']) / 2

# rename strike column for clarity
atm_final_liq = atm_final_liq.rename(columns={'adj_strike_rounded': 'strike', 'DTE_call': 'DTE'})


# keep the core columns for modeling / labeling
selected_straddles = atm_final_liq[[
    'ticker', 'date', 'exdate', 'DTE', 'strike',
    'call_premium', 'put_premium', 'straddle_premium',
    'straddle_iv', 'net_delta',
    'open_interest_call', 'open_interest_put',
    'volume_call', 'volume_put',
    'strike_diff', 'impl_volatility_call', 'impl_volatility_put', 
    'adj_best_bid_call', 'adj_best_offer_call', 
    'adj_best_bid_put', 'adj_best_offer_put'
]].copy()

In [72]:
selected_straddles[selected_straddles['ticker'] == 'AAPL']

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,open_interest_put,volume_call,volume_put,strike_diff,impl_volatility_call,impl_volatility_put,adj_best_bid_call,adj_best_offer_call,adj_best_bid_put,adj_best_offer_put
61268,AAPL,2018-01-02,2018-02-02,31,43.125,4.875,4.950,9.825,0.245063,0.023940,...,370.0,841.0,70.0,0.060003,0.244650,0.245475,4.80,4.95,4.85,5.05
61299,AAPL,2018-01-03,2018-02-02,30,43.125,4.875,5.025,9.900,0.251193,0.017545,...,395.0,1038.0,242.0,0.067500,0.251627,0.250760,4.80,4.95,4.95,5.10
61330,AAPL,2018-01-04,2018-02-02,29,43.125,5.175,4.300,9.475,0.243264,0.075928,...,426.0,566.0,45.0,0.132500,0.246877,0.239650,5.10,5.25,4.25,4.35
61362,AAPL,2018-01-05,2018-02-02,28,43.750,4.900,4.600,9.500,0.245476,0.039665,...,417.0,2105.0,316.0,0.000000,0.248068,0.242885,4.85,4.95,4.55,4.65
61423,AAPL,2018-01-08,2018-02-09,32,43.750,4.775,5.425,10.200,0.239872,-0.016354,...,105.0,530.0,94.0,0.162498,0.242030,0.237714,4.70,4.85,5.35,5.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102533,AAPL,2022-12-23,2023-01-27,35,132.000,6.000,5.600,11.600,0.355586,0.059876,...,278.0,77.0,139.0,0.140000,0.357814,0.353358,5.80,6.20,5.40,5.80
1102597,AAPL,2022-12-27,2023-01-27,31,130.000,6.025,5.450,11.475,0.379205,0.067349,...,2301.0,420.0,1071.0,0.030000,0.383305,0.375106,5.90,6.15,5.40,5.50
1102738,AAPL,2022-12-29,2023-01-27,29,130.000,5.500,5.350,10.850,0.371430,0.040168,...,2336.0,705.0,592.0,0.390000,0.376446,0.366415,5.40,5.60,5.30,5.40
1102813,AAPL,2022-12-30,2023-01-27,28,130.000,5.450,5.225,10.675,0.371711,0.053841,...,2326.0,486.0,363.0,0.070010,0.371279,0.372143,5.40,5.50,5.15,5.30


In [73]:
def trailing_percentile_series(s: pd.Series, window: int) -> pd.Series:
    arr = s.to_numpy()
    out = np.full(len(arr), np.nan, dtype=float)
    for i in range(len(arr)):
        if pd.isna(arr[i]):
            continue
        start = max(0, i - window)
        past = arr[start:i]
        past = past[~np.isnan(past)]  # drop NaNs
        if len(past) == 0:
            out[i] = np.nan
        else:
            out[i] = (past < arr[i]).sum() / len(past)
    return pd.Series(out, index=s.index)

In [74]:
# --- 0. Prepare underlying features on stock_data ---
stock = stock_data.copy()
stock['date'] = pd.to_datetime(stock['date'])
stock['ticker'] = stock['ticker'].astype(str).str.upper()

# log returns
stock['log_ret'] = np.log1p(stock['adj_ret'].fillna(0))

# sort for rolling
stock = stock.sort_values(['ticker', 'date'])

# realized vol (annualized) using transform so index stays aligned
stock['realized_vol_30'] = (
    stock.groupby('ticker')['log_ret']
         .transform(lambda x: x.rolling(30, min_periods=10).std())
    * np.sqrt(252)
)
stock['realized_vol_10'] = (
    stock.groupby('ticker')['log_ret']
         .transform(lambda x: x.rolling(10, min_periods=5).std())
    * np.sqrt(252)
)

# momentum features
stock['mom_5'] = stock.groupby('ticker')['log_ret'].transform(lambda x: x.rolling(5, min_periods=1).sum())
stock['mom_10'] = stock.groupby('ticker')['log_ret'].transform(lambda x: x.rolling(10, min_periods=1).sum())

# realized vol rank over trailing 60 days (excluding current)
stock['realized_vol_30_rank'] = stock.groupby('ticker')['realized_vol_30'] \
                                     .transform(lambda s: trailing_percentile_series(s, 60))


In [75]:
stock

,permno,date,ret,prc,vol,shrout,cfacpr,cfacshr,ticker,adj_prc,adj_ret,adj_vol,adj_shrout,mktcap,log_ret,realized_vol_30,realized_vol_10,mom_5,mom_10,realized_vol_30_rank
0,14593,2018-01-02,0.017905,172.25999,25555934.0,5087056.0,4.0,4.0,AAPL,43.064997,0.017905,25555934.0,20348224.0,8.762962e+08,0.017747,NaN,NaN,0.017747,0.017747,NaN
1,14593,2018-01-03,-0.000174,172.23000,29517900.0,5087056.0,4.0,4.0,AAPL,43.057500,-0.000174,29517900.0,20348224.0,8.761437e+08,-0.000174,NaN,NaN,0.017573,0.017573,NaN
2,14593,2018-01-04,0.004645,173.03000,22434597.0,5087056.0,4.0,4.0,AAPL,43.257500,0.004645,22434597.0,20348224.0,8.802133e+08,0.004634,NaN,NaN,0.022207,0.022207,NaN
3,14593,2018-01-05,0.011385,175.00000,23660018.0,5087056.0,4.0,4.0,AAPL,43.750000,0.011385,23660018.0,20348224.0,8.902348e+08,0.011321,NaN,NaN,0.033528,0.033528,NaN
4,14593,2018-01-08,-0.003714,174.35001,20567766.0,5087056.0,4.0,4.0,AAPL,43.587502,-0.003714,20567766.0,20348224.0,8.869283e+08,-0.003721,NaN,0.137652,0.029807,0.029807,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15085,93436,2023-12-22,-0.007701,252.53999,93148541.0,3178921.0,1.0,1.0,TSLA,252.539990,-0.007701,93148541.0,3178921.0,8.028047e+08,-0.007731,0.383299,0.400951,-0.003794,0.035058,0.0
15086,93436,2023-12-26,0.016116,256.60999,86700724.0,3178921.0,1.0,1.0,TSLA,256.609990,0.016116,86700724.0,3178921.0,8.157429e+08,0.015988,0.381532,0.387764,0.017811,0.068002,0.0
15087,93436,2023-12-27,0.018822,261.44000,106250779.0,3178921.0,1.0,1.0,TSLA,261.440000,0.018822,106250779.0,3178921.0,8.310971e+08,0.018647,0.368693,0.377393,0.016273,0.098102,0.0
15088,93436,2023-12-28,-0.031594,253.17999,113250680.0,3178921.0,1.0,1.0,TSLA,253.179990,-0.031594,113250680.0,3178921.0,8.048392e+08,-0.032104,0.346452,0.432146,0.024146,0.056424,0.0


In [76]:
selected_straddles

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,open_interest_put,volume_call,volume_put,strike_diff,impl_volatility_call,impl_volatility_put,adj_best_bid_call,adj_best_offer_call,adj_best_bid_put,adj_best_offer_put
2,AMD,2018-01-02,2018-02-02,31,11.0,0.750,0.730,1.480,0.579868,0.064396,...,388.0,867.0,46.0,0.02,0.590092,0.569645,0.73,0.77,0.71,0.75
50,AMD,2018-01-04,2018-02-02,29,12.0,0.895,0.790,1.685,0.622124,0.107429,...,266.0,503.0,127.0,0.12,0.624345,0.619902,0.86,0.93,0.76,0.82
74,AMD,2018-01-05,2018-02-02,28,12.0,0.780,0.885,1.665,0.631055,0.028745,...,313.0,790.0,131.0,0.12,0.631549,0.630562,0.77,0.79,0.87,0.90
98,AMD,2018-01-05,2018-02-09,35,12.0,0.815,0.920,1.735,0.588431,0.033836,...,175.0,84.0,121.0,0.12,0.588382,0.588480,0.79,0.84,0.89,0.95
146,AMD,2018-01-09,2018-02-09,31,12.0,0.755,0.920,1.675,0.603827,0.007003,...,226.0,286.0,46.0,0.18,0.603818,0.603837,0.75,0.76,0.90,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598812,RIVN,2022-11-21,2022-12-23,32,28.0,2.800,2.295,5.095,0.764288,0.151125,...,224.0,15.0,17.0,0.42,0.765254,0.763323,2.75,2.85,2.24,2.35
1599103,RIVN,2022-12-02,2022-12-30,28,31.0,2.455,2.030,4.485,0.650289,0.131949,...,160.0,26.0,210.0,0.36,0.649317,0.651261,2.39,2.52,2.00,2.06
1599466,RIVN,2022-12-16,2023-01-20,35,22.5,2.220,2.120,4.340,0.783498,0.103968,...,3679.0,1373.0,1705.0,0.02,0.791373,0.775623,2.15,2.29,2.10,2.14
1599829,RIVN,2022-12-23,2023-01-20,28,19.0,1.840,1.665,3.505,0.833809,0.123657,...,1220.0,237.0,463.0,0.14,0.832646,0.834973,1.82,1.86,1.65,1.68


In [77]:
# --- 1. Target construction: compute exit date and r_long/r_short ---
entries = selected_straddles.copy()
entries['date'] = pd.to_datetime(entries['date']).dt.normalize()
entries['ticker'] = entries['ticker'].astype(str).str.upper()

# Build trading calendar (normalized) and map entry dates to positions
all_dates = pd.Index(pd.to_datetime(options_full['date']).dt.normalize().sort_values().unique())
entries['date_pos'] = all_dates.get_indexer(entries['date'])  # gives -1 if not found
# entries = entries.loc[entries['date_pos'] >= 0].copy()        # drop unmatched dates


In [78]:
entries

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,volume_call,volume_put,strike_diff,impl_volatility_call,impl_volatility_put,adj_best_bid_call,adj_best_offer_call,adj_best_bid_put,adj_best_offer_put,date_pos
2,AMD,2018-01-02,2018-02-02,31,11.0,0.750,0.730,1.480,0.579868,0.064396,...,867.0,46.0,0.02,0.590092,0.569645,0.73,0.77,0.71,0.75,0
50,AMD,2018-01-04,2018-02-02,29,12.0,0.895,0.790,1.685,0.622124,0.107429,...,503.0,127.0,0.12,0.624345,0.619902,0.86,0.93,0.76,0.82,2
74,AMD,2018-01-05,2018-02-02,28,12.0,0.780,0.885,1.665,0.631055,0.028745,...,790.0,131.0,0.12,0.631549,0.630562,0.77,0.79,0.87,0.90,3
98,AMD,2018-01-05,2018-02-09,35,12.0,0.815,0.920,1.735,0.588431,0.033836,...,84.0,121.0,0.12,0.588382,0.588480,0.79,0.84,0.89,0.95,3
146,AMD,2018-01-09,2018-02-09,31,12.0,0.755,0.920,1.675,0.603827,0.007003,...,286.0,46.0,0.18,0.603818,0.603837,0.75,0.76,0.90,0.94,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598812,RIVN,2022-11-21,2022-12-23,32,28.0,2.800,2.295,5.095,0.764288,0.151125,...,15.0,17.0,0.42,0.765254,0.763323,2.75,2.85,2.24,2.35,1231
1599103,RIVN,2022-12-02,2022-12-30,28,31.0,2.455,2.030,4.485,0.650289,0.131949,...,26.0,210.0,0.36,0.649317,0.651261,2.39,2.52,2.00,2.06,1239
1599466,RIVN,2022-12-16,2023-01-20,35,22.5,2.220,2.120,4.340,0.783498,0.103968,...,1373.0,1705.0,0.02,0.791373,0.775623,2.15,2.29,2.10,2.14,1249
1599829,RIVN,2022-12-23,2023-01-20,28,19.0,1.840,1.665,3.505,0.833809,0.123657,...,237.0,463.0,0.14,0.832646,0.834973,1.82,1.86,1.65,1.68,1254


In [79]:

# Horizon H = min(7, DTE - 5); require H >= 1
entries['H'] = np.minimum(7, np.maximum(0, entries['DTE'].astype(int) - 5)).astype(int)
entries = entries.loc[entries['H'] >= 1].copy()

# Compute exit position and enforce bounds
entries['exit_pos'] = entries['date_pos'] + entries['H']
valid_mask = entries['exit_pos'] < len(all_dates)
entries = entries.loc[valid_mask].copy()

# Lookup exit_date from the trading calendar
entries['exit_date'] = all_dates[entries['exit_pos']].to_numpy()

In [80]:
entries

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,impl_volatility_call,impl_volatility_put,adj_best_bid_call,adj_best_offer_call,adj_best_bid_put,adj_best_offer_put,date_pos,H,exit_pos,exit_date
2,AMD,2018-01-02,2018-02-02,31,11.0,0.750,0.730,1.480,0.579868,0.064396,...,0.590092,0.569645,0.73,0.77,0.71,0.75,0,7,7,2018-01-11
50,AMD,2018-01-04,2018-02-02,29,12.0,0.895,0.790,1.685,0.622124,0.107429,...,0.624345,0.619902,0.86,0.93,0.76,0.82,2,7,9,2018-01-16
74,AMD,2018-01-05,2018-02-02,28,12.0,0.780,0.885,1.665,0.631055,0.028745,...,0.631549,0.630562,0.77,0.79,0.87,0.90,3,7,10,2018-01-17
98,AMD,2018-01-05,2018-02-09,35,12.0,0.815,0.920,1.735,0.588431,0.033836,...,0.588382,0.588480,0.79,0.84,0.89,0.95,3,7,10,2018-01-17
146,AMD,2018-01-09,2018-02-09,31,12.0,0.755,0.920,1.675,0.603827,0.007003,...,0.603818,0.603837,0.75,0.76,0.90,0.94,5,7,12,2018-01-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598732,RIVN,2022-11-17,2022-12-16,29,32.5,2.480,2.880,5.360,0.739363,0.036769,...,0.737426,0.741300,2.46,2.50,2.86,2.90,1229,7,1236,2022-11-29
1598754,RIVN,2022-11-18,2022-12-16,28,30.0,2.640,2.275,4.915,0.738708,0.126763,...,0.740020,0.737395,2.62,2.66,2.25,2.30,1230,7,1237,2022-11-30
1598812,RIVN,2022-11-21,2022-12-23,32,28.0,2.800,2.295,5.095,0.764288,0.151125,...,0.765254,0.763323,2.75,2.85,2.24,2.35,1231,7,1238,2022-12-01
1599103,RIVN,2022-12-02,2022-12-30,28,31.0,2.455,2.030,4.485,0.650289,0.131949,...,0.649317,0.651261,2.39,2.52,2.00,2.06,1239,7,1246,2022-12-13


In [81]:
# --- 2. Build exit straddle mid-premiums for lookup ---
# Reconstruct contemporaneous straddle quotes (call+put) across all dates/contracts
opts2 = options_full.copy()
opts2['date'] = pd.to_datetime(opts2['date'])
opts2['exdate'] = pd.to_datetime(opts2['exdate'])
opts2['ticker'] = opts2['ticker'].astype(str).str.upper()

# Need underlying cfacpr to adjust strike if using raw strike_price (assuming stock has cfacpr)
stock_small = stock[['ticker', 'date', 'cfacpr']].drop_duplicates()
opts2 = opts2.merge(stock_small, on=['ticker', 'date'], how='left', validate='many_to_one')
opts2['adj_strike'] = (opts2['strike_price'] / opts2['cfacpr']).round(4)

# mid premium for each leg
opts2['mid_premium'] = (opts2['adj_best_bid'] + opts2['adj_best_offer']) / 2

calls_all = opts2[opts2['cp_flag'].str.upper().isin(['C', 'CALL'])].copy()
puts_all  = opts2[opts2['cp_flag'].str.upper().isin(['P', 'PUT'])].copy()

# join to get straddle quote (must be same ticker/date/exdate/strike)
straddle_quotes = calls_all.merge(
    puts_all,
    on=['ticker', 'date', 'exdate', 'adj_strike'],
    suffixes=('_call', '_put'),
    how='inner'
)

# compute exit value (mid)
straddle_quotes['call_mid'] = (straddle_quotes['adj_best_bid_call'] + straddle_quotes['adj_best_offer_call']) / 2
straddle_quotes['put_mid']  = (straddle_quotes['adj_best_bid_put']  + straddle_quotes['adj_best_offer_put'])  / 2
straddle_quotes['straddle_mid'] = straddle_quotes['call_mid'] + straddle_quotes['put_mid']

# Prepare lookup table for exit
exit_lookup = straddle_quotes[[
    'ticker', 'date', 'exdate', 'adj_strike', 'straddle_mid'
]].rename(columns={'date': 'exit_date', 'adj_strike': 'strike'})

In [82]:
exit_lookup

,ticker,exit_date,exdate,strike,straddle_mid
0,AMD,2018-01-02,2018-01-05,10.0,1.025
1,AMD,2018-01-02,2018-01-05,10.5,0.565
2,AMD,2018-01-02,2018-01-05,11.0,0.350
3,AMD,2018-01-02,2018-01-05,11.5,0.575
4,AMD,2018-01-02,2018-01-05,12.0,1.020
...,...,...,...,...,...
21062128,RIVN,2022-12-30,2025-01-17,65.0,48.920
21062129,RIVN,2022-12-30,2025-01-17,70.0,53.660
21062130,RIVN,2022-12-30,2025-01-17,7.5,15.100
21062131,RIVN,2022-12-30,2025-01-17,75.0,58.265


In [83]:
# --- 3. Merge exit value into entries to get target ---
entries = entries.merge(
    exit_lookup,
    left_on=['ticker', 'exit_date', 'exdate', 'strike'],
    right_on=['ticker', 'exit_date', 'exdate', 'strike'],
    how='left'
)

# drop if exit quote missing
entries = entries.dropna(subset=['straddle_mid']).copy()

# compute target
entries['entry_value'] = entries['straddle_premium']
entries['exit_value'] = entries['straddle_mid']
entries['r_long'] = (entries['exit_value'] - entries['entry_value']) / entries['entry_value']
entries['r_short'] = -entries['r_long']

In [84]:
entries

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,adj_best_offer_put,date_pos,H,exit_pos,exit_date,straddle_mid,entry_value,exit_value,r_long,r_short
0,AMD,2018-01-02,2018-02-02,31,11.0,0.750,0.730,1.480,0.579868,0.064396,...,0.75,0,7,7,2018-01-11,1.670,1.480,1.670,0.128378,-0.128378
1,AMD,2018-01-04,2018-02-02,29,12.0,0.895,0.790,1.685,0.622124,0.107429,...,0.82,2,7,9,2018-01-16,1.350,1.685,1.350,-0.198813,0.198813
2,AMD,2018-01-05,2018-02-02,28,12.0,0.780,0.885,1.665,0.631055,0.028745,...,0.90,3,7,10,2018-01-17,1.380,1.665,1.380,-0.171171,0.171171
3,AMD,2018-01-05,2018-02-09,35,12.0,0.815,0.920,1.735,0.588431,0.033836,...,0.95,3,7,10,2018-01-17,1.510,1.735,1.510,-0.129683,0.129683
4,AMD,2018-01-09,2018-02-09,31,12.0,0.755,0.920,1.675,0.603827,0.007003,...,0.94,5,7,12,2018-01-19,1.605,1.675,1.605,-0.041791,0.041791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10152,RIVN,2022-11-17,2022-12-16,29,32.5,2.480,2.880,5.360,0.739363,0.036769,...,2.90,1229,7,1236,2022-11-29,4.705,5.360,4.705,-0.122201,0.122201
10153,RIVN,2022-11-18,2022-12-16,28,30.0,2.640,2.275,4.915,0.738708,0.126763,...,2.30,1230,7,1237,2022-11-30,4.040,4.915,4.040,-0.178026,0.178026
10154,RIVN,2022-11-21,2022-12-23,32,28.0,2.800,2.295,5.095,0.764288,0.151125,...,2.35,1231,7,1238,2022-12-01,5.200,5.095,5.200,0.020608,-0.020608
10155,RIVN,2022-12-02,2022-12-30,28,31.0,2.455,2.030,4.485,0.650289,0.131949,...,2.06,1239,7,1246,2022-12-13,6.290,4.485,6.290,0.402453,-0.402453


In [85]:
# --- 4. Feature engineering ---

# a) Volatility features
entries['iv_realized_vol_ratio'] = entries['straddle_iv'] / entries.merge(
    stock[['ticker', 'date', 'realized_vol_30']],
    on=['ticker', 'date'],
    how='left'
)['realized_vol_30']


In [86]:
# b) IV percentile rank (trailing 60 days) per ticker for straddle_iv
# iv_rank is a relative measure of how high or low the current implied 
# volatility is compared to its own recent history. 
# In the pipeline it’s the trailing 60-day percentile of the 
# straddle’s implied volatility, excluding the current day.

# ensure sorted so percentiles are trailing
entries = entries.sort_values(['ticker', 'date'])

# correct alignment: use transform, not apply
entries['iv_rank'] = entries.groupby('ticker')['straddle_iv'] \
                           .transform(lambda s: trailing_percentile_series(s, 60))

In [87]:
entries

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,H,exit_pos,exit_date,straddle_mid,entry_value,exit_value,r_long,r_short,iv_realized_vol_ratio,iv_rank
299,AAPL,2018-01-02,2018-02-02,31,43.125,4.875000,4.950000,9.825000,0.245063,0.023940,...,7,7,2018-01-11,8.900000,9.825000,8.900000,-0.094148,0.094148,NaN,NaN
300,AAPL,2018-01-03,2018-02-02,30,43.125,4.875000,5.025000,9.900000,0.251193,0.017545,...,7,8,2018-01-12,9.600000,9.900000,9.600000,-0.030303,0.030303,NaN,1.000000
301,AAPL,2018-01-04,2018-02-02,29,43.125,5.175000,4.300000,9.475000,0.243264,0.075928,...,7,9,2018-01-16,9.660000,9.475000,9.660000,0.019525,-0.019525,NaN,0.000000
302,AAPL,2018-01-05,2018-02-02,28,43.750,4.900000,4.600000,9.500000,0.245476,0.039665,...,7,10,2018-01-17,10.115000,9.500000,10.115000,0.064737,-0.064737,NaN,0.666667
303,AAPL,2018-01-08,2018-02-09,32,43.750,4.775000,5.425000,10.200000,0.239872,-0.016354,...,7,11,2018-01-18,10.850000,10.200000,10.850000,0.063725,-0.063725,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,TSLA,2022-12-15,2023-01-13,29,160.000,10.775000,12.525000,23.300000,0.651948,0.021517,...,7,1255,2022-12-27,51.790000,23.300000,51.790000,1.222747,-1.222747,0.985999,0.833333
6416,TSLA,2022-12-16,2023-01-13,28,150.000,11.675000,10.650000,22.325000,0.673887,0.092289,...,7,1256,2022-12-28,39.130000,22.325000,39.130000,0.752744,-0.752744,1.981167,0.866667
6417,TSLA,2022-12-16,2023-01-20,35,150.000,4.266667,3.908333,8.175000,0.662416,0.100640,...,7,1256,2022-12-28,13.308333,8.175000,13.308333,0.627931,-0.627931,1.957808,0.850000
6418,TSLA,2022-12-19,2023-01-20,32,150.000,4.100000,3.900000,8.000000,0.678836,0.088790,...,7,1257,2022-12-29,10.858333,8.000000,10.858333,0.357292,-0.357292,2.315691,0.866667


In [88]:
# c) Realized vol and its rank from underlying (merge once)
entries = entries.merge(
    stock[['ticker', 'date', 'realized_vol_30', 'realized_vol_10', 'realized_vol_30_rank', 'mom_5', 'mom_10']],
    on=['ticker', 'date'],
    how='left'
)

entries['vol_change'] = entries['realized_vol_30'] - entries['realized_vol_10']



In [89]:
entries.columns

Index(['ticker', 'date', 'exdate', 'DTE', 'strike', 'call_premium',
       'put_premium', 'straddle_premium', 'straddle_iv', 'net_delta',
       'open_interest_call', 'open_interest_put', 'volume_call', 'volume_put',
       'strike_diff', 'impl_volatility_call', 'impl_volatility_put',
       'adj_best_bid_call', 'adj_best_offer_call', 'adj_best_bid_put',
       'adj_best_offer_put', 'date_pos', 'H', 'exit_pos', 'exit_date',
       'straddle_mid', 'entry_value', 'exit_value', 'r_long', 'r_short',
       'iv_realized_vol_ratio', 'iv_rank', 'realized_vol_30',
       'realized_vol_10', 'realized_vol_30_rank', 'mom_5', 'mom_10',
       'vol_change'],
      dtype='object')

In [90]:
# d) Skew & premium
entries['iv_skew'] = entries['impl_volatility_call'] - entries['impl_volatility_put']
entries['log_straddle_premium'] = np.log(entries['straddle_premium'].replace(0, np.nan))


In [91]:
entries.columns

Index(['ticker', 'date', 'exdate', 'DTE', 'strike', 'call_premium',
       'put_premium', 'straddle_premium', 'straddle_iv', 'net_delta',
       'open_interest_call', 'open_interest_put', 'volume_call', 'volume_put',
       'strike_diff', 'impl_volatility_call', 'impl_volatility_put',
       'adj_best_bid_call', 'adj_best_offer_call', 'adj_best_bid_put',
       'adj_best_offer_put', 'date_pos', 'H', 'exit_pos', 'exit_date',
       'straddle_mid', 'entry_value', 'exit_value', 'r_long', 'r_short',
       'iv_realized_vol_ratio', 'iv_rank', 'realized_vol_30',
       'realized_vol_10', 'realized_vol_30_rank', 'mom_5', 'mom_10',
       'vol_change', 'iv_skew', 'log_straddle_premium'],
      dtype='object')

In [92]:
# e) Liquidity / microstructure
# normalized bid-ask spreads for each leg
entries['spread_call'] = (entries['adj_best_offer_call'] - entries['adj_best_bid_call']) / (
    (entries['adj_best_offer_call'] + entries['adj_best_bid_call']) / 2
)
entries['spread_put'] = (entries['adj_best_offer_put'] - entries['adj_best_bid_put']) / (
    (entries['adj_best_offer_put'] + entries['adj_best_bid_put']) / 2
)

# total volume / open interest
entries['total_volume'] = entries['volume_call'] + entries['volume_put']
entries['total_oi'] = entries['open_interest_call'] + entries['open_interest_put']



In [93]:
# --- Prepare full history of open interest for the relevant strikes ---
# Normalize tickers/dates and compute adjusted strike in options_full
opts_full = options_full.copy()
opts_full['date'] = pd.to_datetime(opts_full['date'])
opts_full['exdate'] = pd.to_datetime(opts_full['exdate'])
opts_full['ticker'] = opts_full['ticker'].astype(str).str.upper()

# Merge cfacpr from stock to adjust strike (same logic as before)
stock_small = stock_data[['ticker', 'date', 'cfacpr']].copy()
stock_small['ticker'] = stock_small['ticker'].astype(str).str.upper()
stock_small['date'] = pd.to_datetime(stock_small['date'])

opts_full = opts_full.merge(
    stock_small,
    on=['ticker', 'date'],
    how='left',
    validate='many_to_one'
)

# Compute adjusted strike and round to match entries
opts_full['adj_strike'] = (opts_full['strike_price'] / opts_full['cfacpr']).round(4)

# Separate call and put histories
calls_full = opts_full[opts_full['cp_flag'].str.upper().isin(['C', 'CALL'])].copy()
puts_full  = opts_full[opts_full['cp_flag'].str.upper().isin(['P', 'PUT'])].copy()

# Compute lagged open interest (5 trading days) per contract (ticker, exdate, strike)
calls_full = calls_full.sort_values(['ticker', 'exdate', 'adj_strike', 'date'])
puts_full  = puts_full.sort_values(['ticker', 'exdate', 'adj_strike', 'date'])

calls_full['oi_call_lag5_full'] = calls_full.groupby(['ticker', 'exdate', 'adj_strike'])['open_interest'].shift(5)
puts_full['oi_put_lag5_full']   = puts_full.groupby(['ticker', 'exdate', 'adj_strike'])['open_interest'].shift(5)

# Rename for clarity before merging
calls_full = calls_full.rename(columns={'open_interest': 'open_interest_call_full'})
puts_full  = puts_full.rename(columns={'open_interest': 'open_interest_put_full'})

# --- Merge the full-history open interest (and its 5-day lag) into entries ---
# entries['strike'] should match adj_strike rounding
entries = entries.merge(
    calls_full[[
        'ticker', 'date', 'exdate', 'adj_strike',
        'open_interest_call_full', 'oi_call_lag5_full'
    ]].rename(columns={'adj_strike': 'strike'}),
    on=['ticker', 'date', 'exdate', 'strike'],
    how='left'
)

entries = entries.merge(
    puts_full[[
        'ticker', 'date', 'exdate', 'adj_strike',
        'open_interest_put_full', 'oi_put_lag5_full'
    ]].rename(columns={'adj_strike': 'strike'}),
    on=['ticker', 'date', 'exdate', 'strike'],
    how='left'
)

# --- Compute week-over-week changes using the full-history values ---
entries['oi_change_call'] = (
    (entries['open_interest_call_full'] - entries['oi_call_lag5_full']) /
    entries['oi_call_lag5_full']
)
entries['oi_change_put'] = (
    (entries['open_interest_put_full'] - entries['oi_put_lag5_full']) /
    entries['oi_put_lag5_full']
)


In [94]:
entries.head(40)

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,spread_call,spread_put,total_volume,total_oi,open_interest_call_full,oi_call_lag5_full,open_interest_put_full,oi_put_lag5_full,oi_change_call,oi_change_put
0,AAPL,2018-01-02,2018-02-02,31,43.125,4.875,4.950,9.825,0.245063,0.023940,...,0.030769,0.040404,911.0,3017.0,2647.0,NaN,370.0,NaN,NaN,NaN
1,AAPL,2018-01-03,2018-02-02,30,43.125,4.875,5.025,9.900,0.251193,0.017545,...,0.030769,0.029851,1280.0,3049.0,2654.0,NaN,395.0,NaN,NaN,NaN
2,AAPL,2018-01-04,2018-02-02,29,43.125,5.175,4.300,9.475,0.243264,0.075928,...,0.028986,0.023256,611.0,3283.0,2857.0,NaN,426.0,NaN,NaN,NaN
3,AAPL,2018-01-05,2018-02-02,28,43.750,4.900,4.600,9.500,0.245476,0.039665,...,0.020408,0.021739,2421.0,3232.0,2815.0,NaN,417.0,NaN,NaN,NaN
4,AAPL,2018-01-08,2018-02-09,32,43.750,4.775,5.425,10.200,0.239872,-0.016354,...,0.031414,0.027650,624.0,1275.0,1170.0,NaN,105.0,NaN,NaN,NaN
5,AAPL,2018-01-09,2018-02-09,31,43.750,4.650,5.275,9.925,0.236897,-0.020468,...,0.021505,0.009479,1110.0,1670.0,1514.0,87.0,156.0,52.0,16.402299,2.000000
6,AAPL,2018-01-10,2018-02-09,30,43.750,4.500,5.400,9.900,0.240133,-0.028271,...,0.022222,0.018519,871.0,2263.0,2095.0,614.0,168.0,55.0,2.412052,2.054545
7,AAPL,2018-01-11,2018-02-09,29,43.750,4.775,4.675,9.450,0.232196,0.037014,...,0.031414,0.032086,895.0,2899.0,2514.0,803.0,385.0,70.0,2.130760,4.500000
8,AAPL,2018-01-12,2018-02-16,35,43.750,6.050,4.300,10.350,0.232419,0.146494,...,0.016529,0.023256,10042.0,64199.0,52140.0,46530.0,12059.0,9457.0,0.120567,0.275140
9,AAPL,2018-01-16,2018-02-16,31,43.750,5.900,4.700,10.600,0.255856,0.093241,...,0.016949,0.021277,11593.0,65068.0,51974.0,46649.0,13094.0,9842.0,0.114150,0.330421


In [95]:
# # week-over-week change in open interest per contract (lag 5 trading days)
# entries = entries.sort_values(['ticker', 'exdate', 'strike', 'date'])
# entries['oi_call_lag5'] = entries.groupby(['ticker', 'exdate', 'strike'])['open_interest_call'].shift(5)
# entries['oi_put_lag5']  = entries.groupby(['ticker', 'exdate', 'strike'])['open_interest_put'].shift(5)
# entries['oi_change_call'] = (entries['open_interest_call'] - entries['oi_call_lag5']) / entries['oi_call_lag5']
# entries['oi_change_put']  = (entries['open_interest_put']  - entries['oi_put_lag5'])  / entries['oi_put_lag5']


In [96]:
# f) Temporal
# normalize DTE by its 95th percentile (global)
max_dte = entries['DTE'].quantile(0.95)
entries['DTE_norm'] = entries['DTE'] / max_dte
entries['day_of_week'] = entries['date'].dt.dayofweek  # 0=Monday

In [97]:
entries.head(30)

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,total_volume,total_oi,open_interest_call_full,oi_call_lag5_full,open_interest_put_full,oi_put_lag5_full,oi_change_call,oi_change_put,DTE_norm,day_of_week
0,AAPL,2018-01-02,2018-02-02,31,43.125,4.875,4.950,9.825,0.245063,0.023940,...,911.0,3017.0,2647.0,NaN,370.0,NaN,NaN,NaN,0.885714,1
1,AAPL,2018-01-03,2018-02-02,30,43.125,4.875,5.025,9.900,0.251193,0.017545,...,1280.0,3049.0,2654.0,NaN,395.0,NaN,NaN,NaN,0.857143,2
2,AAPL,2018-01-04,2018-02-02,29,43.125,5.175,4.300,9.475,0.243264,0.075928,...,611.0,3283.0,2857.0,NaN,426.0,NaN,NaN,NaN,0.828571,3
3,AAPL,2018-01-05,2018-02-02,28,43.750,4.900,4.600,9.500,0.245476,0.039665,...,2421.0,3232.0,2815.0,NaN,417.0,NaN,NaN,NaN,0.800000,4
4,AAPL,2018-01-08,2018-02-09,32,43.750,4.775,5.425,10.200,0.239872,-0.016354,...,624.0,1275.0,1170.0,NaN,105.0,NaN,NaN,NaN,0.914286,0
5,AAPL,2018-01-09,2018-02-09,31,43.750,4.650,5.275,9.925,0.236897,-0.020468,...,1110.0,1670.0,1514.0,87.0,156.0,52.0,16.402299,2.000000,0.885714,1
6,AAPL,2018-01-10,2018-02-09,30,43.750,4.500,5.400,9.900,0.240133,-0.028271,...,871.0,2263.0,2095.0,614.0,168.0,55.0,2.412052,2.054545,0.857143,2
7,AAPL,2018-01-11,2018-02-09,29,43.750,4.775,4.675,9.450,0.232196,0.037014,...,895.0,2899.0,2514.0,803.0,385.0,70.0,2.130760,4.500000,0.828571,3
8,AAPL,2018-01-12,2018-02-16,35,43.750,6.050,4.300,10.350,0.232419,0.146494,...,10042.0,64199.0,52140.0,46530.0,12059.0,9457.0,0.120567,0.275140,1.000000,4
9,AAPL,2018-01-16,2018-02-16,31,43.750,5.900,4.700,10.600,0.255856,0.093241,...,11593.0,65068.0,51974.0,46649.0,13094.0,9842.0,0.114150,0.330421,0.885714,1


In [98]:
# ================== Training ==================

In [99]:
entries.columns

Index(['ticker', 'date', 'exdate', 'DTE', 'strike', 'call_premium',
       'put_premium', 'straddle_premium', 'straddle_iv', 'net_delta',
       'open_interest_call', 'open_interest_put', 'volume_call', 'volume_put',
       'strike_diff', 'impl_volatility_call', 'impl_volatility_put',
       'adj_best_bid_call', 'adj_best_offer_call', 'adj_best_bid_put',
       'adj_best_offer_put', 'date_pos', 'H', 'exit_pos', 'exit_date',
       'straddle_mid', 'entry_value', 'exit_value', 'r_long', 'r_short',
       'iv_realized_vol_ratio', 'iv_rank', 'realized_vol_30',
       'realized_vol_10', 'realized_vol_30_rank', 'mom_5', 'mom_10',
       'vol_change', 'iv_skew', 'log_straddle_premium', 'spread_call',
       'spread_put', 'total_volume', 'total_oi', 'open_interest_call_full',
       'oi_call_lag5_full', 'open_interest_put_full', 'oi_put_lag5_full',
       'oi_change_call', 'oi_change_put', 'DTE_norm', 'day_of_week'],
      dtype='object')

In [100]:
entries.to_csv('Straddle_entries_with_features.csv', index = False)

In [6]:
# ================ Features Construction Complete, Start Here ================

In [7]:
entries = pd.read_csv('Straddle_entries_with_features.csv')

In [13]:
entries

,ticker,date,exdate,DTE,strike,call_premium,put_premium,straddle_premium,straddle_iv,net_delta,...,total_volume,total_oi,DTE_norm,day_of_week,open_interest_call_full,oi_call_lag5_full,open_interest_put_full,oi_put_lag5_full,oi_change_call,oi_change_put
0,AAPL,2018-01-02,2018-02-02,31,43.125,4.875000,4.950000,9.825000,0.245062,0.023940,...,911.0,3017.0,0.885714,1,2647.0,NaN,370.0,NaN,NaN,NaN
1,AAPL,2018-01-03,2018-02-02,30,43.125,4.875000,5.025000,9.900000,0.251193,0.017545,...,1280.0,3049.0,0.857143,2,2654.0,NaN,395.0,NaN,NaN,NaN
2,AAPL,2018-01-04,2018-02-02,29,43.125,5.175000,4.300000,9.475000,0.243263,0.075928,...,611.0,3283.0,0.828571,3,2857.0,NaN,426.0,NaN,NaN,NaN
3,AAPL,2018-01-05,2018-02-02,28,43.750,4.900000,4.600000,9.500000,0.245476,0.039665,...,2421.0,3232.0,0.800000,4,2815.0,NaN,417.0,NaN,NaN,NaN
4,AAPL,2018-01-08,2018-02-09,32,43.750,4.775000,5.425000,10.200000,0.239872,-0.016354,...,624.0,1275.0,0.914286,0,1170.0,NaN,105.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10511,TSLA,2022-12-15,2023-01-13,29,160.000,10.775000,12.525000,23.300000,0.651948,0.021517,...,3764.0,1827.0,0.828571,3,1085.0,9.0,742.0,182.0,119.555556,3.076923
10512,TSLA,2022-12-09,2023-01-13,35,180.000,13.525000,13.625000,27.150000,0.613104,0.067287,...,430.0,451.0,1.000000,4,221.0,0.0,230.0,39.0,inf,4.897436
10513,TSLA,2022-12-20,2023-01-20,31,136.670,4.275000,3.641667,7.916667,0.745021,0.127413,...,1959.0,5125.0,0.885714,1,1960.0,1852.0,3165.0,3083.0,0.058315,0.026597
10514,TSLA,2022-12-16,2023-01-20,35,150.000,4.266667,3.908333,8.175000,0.662416,0.100640,...,23687.0,51855.0,1.000000,4,8275.0,5979.0,43580.0,37528.0,0.384011,0.161266


In [8]:
# ---------- helpers ----------
def make_walk_forward_splits(df, date_col="date", initial_train_months=6, val_months=1):
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df["ym"] = df[date_col].dt.to_period("M")
    months = sorted(df["ym"].unique())
    for i in range(initial_train_months, len(months) - val_months + 1):
        train_months = months[:i]
        val_months_list = months[i : i + val_months]
        train_idx = df[df["ym"].isin(train_months)].index
        val_idx = df[df["ym"].isin(val_months_list)].index
        yield train_idx, val_idx, train_months, val_months_list

def tune_thresholds(preds, actual_r, grid_size=20):
    high = np.quantile(np.abs(preds), 0.95)
    candidate_deltas = np.linspace(0, high, grid_size)[1:]
    best_long, best_short = 0.0, 0.0
    best_metric = -np.inf
    for dl in candidate_deltas:
        for ds in candidate_deltas:
            long_mask = preds > dl
            short_mask = preds < -ds
            returns = np.zeros_like(preds)
            returns[long_mask] = actual_r[long_mask]
            returns[short_mask] = -actual_r[short_mask]
            total_return = returns.sum()
            if total_return > best_metric:
                best_metric = total_return
                best_long, best_short = dl, ds
    return best_long, best_short

def simulate_strategy(preds, actual_r, delta_long, delta_short):
    long_mask = preds > delta_long
    short_mask = preds < -delta_short
    signals = np.zeros_like(preds, dtype=int)  # 1=long, -1=short, 0=skip
    signals[long_mask] = 1
    signals[short_mask] = -1
    returns = np.zeros_like(preds)
    returns[long_mask] = actual_r[long_mask]
    returns[short_mask] = -actual_r[short_mask]
    num_trades = (long_mask | short_mask).sum()
    win_rate = ((returns > 0) & (signals != 0)).sum() / max(1, num_trades)
    avg_return = returns.sum() / max(1, num_trades) if num_trades > 0 else 0.0
    vol = returns[signals != 0].std(ddof=0) if num_trades > 0 else np.nan
    sharpe = avg_return / vol if vol and vol > 0 else np.nan
    total_return = returns.sum()
    return {
        "delta_long": delta_long,
        "delta_short": delta_short,
        "num_trades": int(num_trades),
        "win_rate": win_rate,
        "avg_return_per_trade": avg_return,
        "sharpe_per_trade": sharpe,
        "total_return": total_return,
        "returns_vector": returns,
        "signals": signals,
    }

In [9]:
# ---------- prepare model_df ----------
df = entries.copy()
df['date'] = pd.to_datetime(df['date'])
df['ticker'] = df['ticker'].astype(str).str.upper()
df = df.dropna(subset=['r_long'])

feature_columns = [
    'straddle_iv', 'iv_realized_vol_ratio', 'iv_rank',
    'realized_vol_30', 'realized_vol_30_rank',
    'vol_change', 'mom_5', 'mom_10',
    'iv_skew', 'log_straddle_premium',
    'spread_call', 'spread_put',
    'total_volume', 'total_oi',
    'oi_change_call', 'oi_change_put',
    'DTE_norm', 'day_of_week', 'net_delta'
]
feature_columns = [c for c in feature_columns if c in df.columns]

model_df = df[feature_columns + ['r_long', 'date']].copy()
model_df = model_df.dropna(subset=feature_columns + ['r_long']).reset_index(drop=True)

In [10]:
model_df

,straddle_iv,iv_realized_vol_ratio,iv_rank,realized_vol_30,realized_vol_30_rank,vol_change,mom_5,mom_10,iv_skew,log_straddle_premium,...,spread_put,total_volume,total_oi,oi_change_call,oi_change_put,DTE_norm,day_of_week,net_delta,r_long,date
0,0.266059,2.131436,1.000000,0.124826,1.000000,0.013344,0.026994,0.038939,0.005910,2.404690,...,0.024691,16228.0,86641.0,0.474753,4.100100,0.857143,2,-0.032635,0.153047,2018-01-17
1,0.267612,2.218855,1.000000,0.120608,0.500000,0.010091,0.028116,0.040006,0.002043,2.393339,...,0.025105,14407.0,87972.0,0.435659,4.315599,0.828571,3,-0.024400,0.416895,2018-01-18
2,0.255910,2.089476,0.833333,0.122476,0.666667,0.004235,0.017979,0.030898,-0.000096,2.336987,...,0.016260,18687.0,87693.0,0.374562,4.371646,0.800000,4,-0.086112,0.550242,2018-01-19
3,0.285643,1.814257,1.000000,0.157443,1.000000,-0.015534,-0.046531,-0.018415,0.016779,2.397895,...,0.030151,584.0,2011.0,0.146310,0.781701,0.828571,3,0.079158,0.357727,2018-01-25
4,0.282279,1.846500,0.944444,0.152872,0.875000,-0.016837,-0.039723,-0.021744,0.000302,2.379546,...,0.032787,430.0,1504.0,0.575053,0.545825,0.800000,4,-0.048722,0.102315,2018-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10137,0.651948,0.985999,0.833333,0.632243,0.766667,0.134728,-0.095327,-0.210955,0.006036,3.148453,...,0.011976,3764.0,1827.0,119.555556,3.076923,0.828571,3,0.021517,1.222747,2022-12-15
10138,0.613104,0.963026,0.533333,0.621269,0.666667,0.037186,-0.084615,-0.021055,0.009543,3.301377,...,0.025688,430.0,451.0,inf,4.897436,1.000000,4,0.067287,0.616390,2022-12-09
10139,0.745021,2.541462,1.000000,0.661206,0.933333,0.104794,-0.155289,-0.266151,0.020833,2.068970,...,0.004577,1959.0,5125.0,0.058315,0.026597,0.885714,1,0.127413,-0.032632,2022-12-20
10140,0.662416,1.957808,0.850000,0.640575,0.866667,0.142496,-0.175498,-0.260113,0.016402,2.101081,...,0.004264,23687.0,51855.0,0.384011,0.161266,1.000000,4,0.100640,0.627931,2022-12-16


In [11]:


# ---------- walk-forward training ----------
INITIAL_TRAIN_MONTHS = 6
VALIDATION_HORIZON_MONTHS = 1
THRESHOLD_GRID_SIZE = 15
MIN_TRAIN_ROWS = 500

results = []
for train_idx, val_idx, train_months, val_months in make_walk_forward_splits(
    model_df, date_col='date', initial_train_months=INITIAL_TRAIN_MONTHS, val_months=VALIDATION_HORIZON_MONTHS
):
    if len(train_idx) < MIN_TRAIN_ROWS or len(val_idx) == 0:
        continue

    train_df = model_df.loc[train_idx].copy()
    val_df = model_df.loc[val_idx].copy()

    train_df = train_df.dropna(subset=feature_columns + ['r_long'])
    val_df = val_df.dropna(subset=feature_columns + ['r_long'])
    if train_df.empty or val_df.empty:
        continue

    X_train = train_df[feature_columns].to_numpy()
    X_val = val_df[feature_columns].to_numpy()
    y_train = train_df['r_long'].to_numpy()
    y_val = val_df['r_long'].to_numpy()

    # train-scoring model (HistGradientBoostingRegressor handles unscaled input reasonably)
    model = HistGradientBoostingRegressor(
        random_state=42,
        max_iter=300,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=30,
    )
    model.fit(X_train, y_train)

    preds_val = model.predict(X_val)

    delta_long, delta_short = tune_thresholds(preds_val, y_val, grid_size=THRESHOLD_GRID_SIZE)
    stats = simulate_strategy(preds_val, y_val, delta_long, delta_short)
    stats.update({
        "train_period": (train_months[0].to_timestamp(), train_months[-1].to_timestamp()),
        "val_period": (val_months[0].to_timestamp(),),
        "model": model,
        "delta_long": delta_long,
        "delta_short": delta_short,
    })

    baseline = simulate_strategy(preds_val, y_val, delta_long=0, delta_short=0)
    stats["baseline_total_return"] = baseline["total_return"]
    stats["baseline_sharpe"] = baseline["sharpe_per_trade"]

    results.append(stats)

/Applications/anaconda3/envs/env_opt/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Applications/anaconda3/envs/env_opt/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Applications/anaconda3/envs/env_opt/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Applications/anaconda3/envs/env_opt/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Applications/anaconda3/envs/env_opt/lib/python3.9/site-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Applications/anaconda3/envs/env_opt/lib/python3.9/site-packages/numpy

In [12]:
# ---------- summarize ----------
summary_rows = []
for r in results:
    summary_rows.append({
        "train_period": f"{r['train_period'][0].strftime('%Y-%m')} -> {r['train_period'][1].strftime('%Y-%m')}",
        "val_period": f"{r['val_period'][0].strftime('%Y-%m')}",
        "total_return": r["total_return"],
        "avg_return_per_trade": r["avg_return_per_trade"],
        "sharpe": r["sharpe_per_trade"],
        "win_rate": r["win_rate"],
        "num_trades": r["num_trades"],
        "delta_long": r["delta_long"],
        "delta_short": r["delta_short"],
        "baseline_return": r["baseline_total_return"],
        "baseline_sharpe": r["baseline_sharpe"],
    })
summary_df = pd.DataFrame(summary_rows)
print("Fold-level performance:")
print(summary_df)
print("\nAggregate metrics:")
print("Total return (sum over folds):", summary_df["total_return"].sum())
print("Mean Sharpe:", summary_df["sharpe"].mean())

Fold-level performance:
          train_period val_period  total_return  avg_return_per_trade  \
0   2018-01 -> 2018-06    2018-07      7.195704              0.087752   
1   2018-01 -> 2018-07    2018-08     16.773590              0.246670   
2   2018-01 -> 2018-08    2018-09      2.842366              0.047373   
3   2018-01 -> 2018-09    2018-10      4.297656              0.075397   
4   2018-01 -> 2018-10    2018-11      0.391389              0.030107   
5   2018-01 -> 2018-11    2018-12      2.915307              0.224254   
6   2018-01 -> 2018-12    2019-01      0.821277              0.023465   
7   2018-01 -> 2019-01    2019-02      2.071105              0.059174   
8   2018-01 -> 2019-02    2019-03      6.672115              0.083401   
9   2018-01 -> 2019-03    2019-04      3.270837              0.033039   
10  2018-01 -> 2019-04    2019-05     12.832457              0.116659   
11  2018-01 -> 2019-05    2019-06      1.996907              0.048705   
12  2018-01 -> 2019-06    2

In [47]:
# ================ Test on 23 ================

In [49]:
stock_data_1_10 = pd.read_csv("stock_data_2018_2023_top10_roots.csv")
stock_data_11_20 = pd.read_csv("stock_data_2018_2023_top_11_20.csv")
stock_prc = pd.read_csv('openprice_and_adjprc_cleaned.csv')
# Convert column names of stock_data_11_20 to lowercase
stock_data_11_20.columns = stock_data_11_20.columns.str.lower()

# Find common columns between the two dataframes
common_columns = stock_data_1_10.columns.intersection(stock_data_11_20.columns)

# Concatenate on common columns
stock_data_23 = pd.concat(
    [stock_data_1_10[common_columns], stock_data_11_20[common_columns]],
    ignore_index=True
)

stock_data_23 = pd.merge(stock_data_23, stock_prc, on = ['ticker', 'date'], how = 'left')
stock_data_23['adj_prc'] = stock_data_23['close_price']

stock_data_23['date'] = pd.to_datetime(stock_data_23['date'])
stock_data_23.to_csv('stock_1824_top20.csv', index = False)

In [50]:
stock_data_23

,permno,date,cfacpr,ticker,open_price,close_price,adj_prc
0,14593,2018-01-02,4.0,AAPL,42.540001,43.064999,43.064999
1,14593,2018-01-03,4.0,AAPL,43.132500,43.057499,43.057499
2,14593,2018-01-04,4.0,AAPL,43.134998,43.257500,43.257500
3,14593,2018-01-05,4.0,AAPL,43.360001,43.750000,43.750000
4,14593,2018-01-08,4.0,AAPL,43.587502,43.587502,43.587502
...,...,...,...,...,...,...,...
29300,93002,2024-12-24,1.0,AVGO,240.119995,239.679993,239.679993
29301,93002,2024-12-26,1.0,AVGO,240.000000,245.360001,245.360001
29302,93002,2024-12-27,1.0,AVGO,245.639999,241.750000,241.750000
29303,93002,2024-12-30,1.0,AVGO,234.699997,235.580002,235.580002
